In [ ]:
!pip install huggingface_hub
!huggingface-cli login

In [ ]:
from huggingface_hub import login
login(token = "hf_key")

In [ ]:
!git config --global credential.helper store

In [ ]:
# makeing repo
!huggingface-cli repo create OrphanSimBERT

In [ ]:
# deleting repo
#!pip install huggingface_hub
from huggingface_hub import HfApi
api = HfApi()
api.delete_repo(repo_id='jabv/OrphanSimBERT')

#Example finetuuning

In [ ]:
!pip install transformers sentence-transformers
#from datasets import load_dataset
from sentence_transformers import SentenceTransformer, models
from transformers import BertTokenizer
from transformers import get_linear_schedule_with_warmup
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader
from tqdm import tqdm
import time
import datetime
import random
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModel
from transformers import BertTokenizer, RobertaTokenizer, RobertaModel
tokenizer = AutoTokenizer.from_pretrained('microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext')
class BertForSTS(torch.nn.Module):
    def __init__(self):
        super(BertForSTS, self).__init__()
        #self.bert = models.Transformer("bert-base-uncased", max_seq_length=512)
        self.bert = models.Transformer('microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext', max_seq_length=1000)
        #model = AutoModel.from_pretrained(model_name)
        self.pooling_layer = models.Pooling(self.bert.get_word_embedding_dimension())
        self.sts_bert = SentenceTransformer(modules=[self.bert, self.pooling_layer])
    def forward(self, input_data):
        output = self.sts_bert(input_data)['sentence_embedding']
        return output
# Instantiate the model and move it to GPU
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

model = BertForSTS()
model.to(device)
model.eval()
def predict_similarity(sentence_pair):
  test_input = tokenizer(sentence_pair, padding=True, truncation=True, return_tensors="pt").to(device) # max_length = 512,
  test_input['input_ids'] = test_input['input_ids']
  test_input['attention_mask'] = test_input['attention_mask']
  del test_input['token_type_ids']
  output = model(test_input)
  sim = torch.nn.functional.cosine_similarity(output[0], output[1], dim=0).item()
  return sim

from google.colab import drive
drive.mount('/content/drive')
PATH = '/content/drive/My Drive/Finetunned_Bert_2.pt'
model = BertForSTS()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.load_state_dict(torch.load(PATH, map_location=device))
model.eval()

print("##########################################################################################")
example_1=[ 'A cat is walking around a house.', 'A woman is peeling potato']
print(f"Sentence 1: {example_1[0]}")
print(f"Sentence 2: {example_1[1]}")
print(f"Predicted similarity score: {round(predict_similarity(example_1), 2)}")

print("##########################################################################################")
example_1=[  'arterial thoracic outlet syndrome',
 '''
 The objective of this project is to study the contribution of hemorrheology to arterial and venous thrombosis using the arterio-venous (A-V)
   ''']
print(f"Sentence 1: {example_1[0]}")
print(f"Sentence 2: {example_1[1]}")
print(f"Predicted similarity score: {round(predict_similarity(example_1), 2)}")


print("##########################################################################################")
example_1=['antisynthetase syndrome',
 '''
  A rare idiopathic inflammatory myopathy (IIM) historically characterized by symmetric proximal muscle weakness, elevated muscle enzymes (creatine kinase), myopathic findings on electromyography, and muscle biopsy showing endomyial infiltration composed mainly of macrophages and lymphocytes. The features are non-specific, thus the disease should be distinguished from similar entities with specific clinical, immunological, histological features, notably dermatomyositis, immune-mediated necrotizing myopathy, anti-synthetase syndrome, inclusion body myositis, and myositis associated with other connective tissue disorder.
''']
print(f"Sentence 1: {example_1[0]}")
print(f"Sentence 2: {example_1[1]}")
print(f"Predicted similarity score: {round(predict_similarity(example_1), 2)}")

print("##########################################################################################")
##(one of its synonym is childhood cancer)
example_1=['mismatch repair cancer syndrome 1 '
,
'''
childhood cancer''']

print(f"Sentence 1: {example_1[0]}")
print(f"Sentence 2: {example_1[1]}")
print(f"Predicted similarity score: {round(predict_similarity(example_1), 2)}")

print("##########################################################################################")
example_1=[  'polymyositis',
 '''
 A rare idiopathic inflammatory myopathy (IIM) historically characterized by symmetric proximal muscle weakness, elevated muscle enzymes (creatine kinase), myopathic findings on electromyography, and muscle biopsy showing endomyial infiltration composed mainly of macrophages and lymphocytes. The features are non-specific, thus the disease should be distinguished from similar entities with specific clinical, immunological, histological features, notably dermatomyositis, immune-mediated necrotizing myopathy, anti-synthetase syndrome, inclusion body myositis, and myositis associated with other connective tissue disorder.
   '''
]
print(f"Sentence 1: {example_1[0]}")
print(f"Sentence 2: {example_1[1]}")
print(f"Predicted similarity score: {round(predict_similarity(example_1), 2)}")

def is_about_term(input_text, target_term):
  sentence_pair=[input_text, target_term]
  test_input = tokenizer(sentence_pair, padding=True, truncation=True, return_tensors="pt").to(device) # max_length = 512,
  test_input['input_ids'] = test_input['input_ids']
  test_input['attention_mask'] = test_input['attention_mask']
  del test_input['token_type_ids']
  output = model(test_input)
  sim = torch.nn.functional.cosine_similarity(output[0], output[1], dim=0).item()
  return  round(sim,2)

# Pusing model and tokenizer to repo

In [ ]:
class BertForSTS(torch.nn.Module):
    def __init__(self):
        super(BertForSTS, self).__init__()
        #self.bert = models.Transformer("bert-base-uncased", max_seq_length=512)
        self.bert = models.Transformer('microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext', max_seq_length=1000)
        #model = AutoModel.from_pretrained(model_name)
        self.pooling_layer = models.Pooling(self.bert.get_word_embedding_dimension())
        self.sts_bert = SentenceTransformer(modules=[self.bert, self.pooling_layer])
    def forward(self, input_data):
        output = self.sts_bert(input_data)['sentence_embedding']
        return output
# Instantiate the model and move it to GPU
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")
#drive.mount('/content/drive')
PATH = '/content/drive/My Drive/Finetunned_Bert_2.pt'
model = BertForSTS()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.load_state_dict(torch.load(PATH, map_location=device))


No GPU available, using the CPU instead.


<All keys matched successfully>

In [ ]:
!huggingface-cli repo create OrphanSimBERT_1

In [ ]:
import os
import torch
from huggingface_hub import HfApi, HfFolder, create_repo, upload_file
from transformers import AutoTokenizer

# Set your Hugging Face token
HF_TOKEN = "hf_key"

# Save the model
model_dir = "OrphanSimBERT_1"
os.makedirs(model_dir, exist_ok=True)
model_path = os.path.join(model_dir, "OrphanSimBERT.bin")
torch.save(model.state_dict(), model_path)

# Save tokenizer
tokenizer = AutoTokenizer.from_pretrained('microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext')
tokenizer.save_pretrained(model_dir)

# Save the model config
config = {
    "architectures": ["BertForSTS"],
    "model_type": "bert",
    "tokenizer_class": "BertTokenizer",
    "pretrained_model_name_or_path": "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext"
}
config_path = os.path.join(model_dir, "config.json")
with open(config_path, 'w') as f:
    import json
    json.dump(config, f)

# Create the repository on Hugging Face
repo_name = "OrphanSimBERT_1"
username = HfApi().whoami(token=HF_TOKEN)["name"]  # Your username on Hugging Face
repo_id = f"{username}/{repo_name}"

api = HfApi()
create_repo(repo_id, token=HF_TOKEN, exist_ok=True)

# Upload files to the repository
upload_file(
    path_or_fileobj=model_path,
    path_in_repo="pytorch_model.bin",
    repo_id=repo_id,
    token=HF_TOKEN
)
upload_file(
    path_or_fileobj=config_path,
    path_in_repo="config.json",
    repo_id=repo_id,
    token=HF_TOKEN
)
upload_file(
    path_or_fileobj=os.path.join(model_dir, "tokenizer_config.json"),
    path_in_repo="tokenizer_config.json",
    repo_id=repo_id,
    token=HF_TOKEN
)
upload_file(
    path_or_fileobj=os.path.join(model_dir, "vocab.txt"),
    path_in_repo="vocab.txt",
    repo_id=repo_id,
    token=HF_TOKEN
)

print("Model and tokenizer files have been uploaded to the repository.")

OrphanSimBERT.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Model and tokenizer files have been uploaded to the repository.


In [ ]:
from huggingface_hub import HfApi
api = HfApi()
api.delete_repo(repo_id='jabv/OrphanSimBERT_1')

In [ ]:
import os
import torch
from sentence_transformers import SentenceTransformer, models
from huggingface_hub import HfApi, HfFolder, create_repo, upload_file
from transformers import AutoTokenizer

class BertForSTS(torch.nn.Module):
    def __init__(self):
        super(BertForSTS, self).__init__()
        self.bert = models.Transformer('microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext', max_seq_length=1000)
        self.pooling_layer = models.Pooling(self.bert.get_word_embedding_dimension())
        self.sts_bert = SentenceTransformer(modules=[self.bert, self.pooling_layer])

    def forward(self, input_data):
        output = self.sts_bert(input_data)['sentence_embedding']
        return output

# Instantiate the model and move it to GPU
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

# Load the fine-tuned model
PATH = '/content/drive/My Drive/Finetunned_Bert_2.pt'
model = BertForSTS()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.load_state_dict(torch.load(PATH, map_location=device))

# Set your Hugging Face token
HF_TOKEN = os.getenv('hf_key')  # Ensure you set this environment variable securely

# Save the model
model_dir = "OrphanSimBERT_1"
os.makedirs(model_dir, exist_ok=True)
model_path = os.path.join(model_dir, "OrphanSimBERT.bin")
torch.save(model.state_dict(), model_path)

# Save tokenizer
tokenizer = AutoTokenizer.from_pretrained('microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext')
tokenizer.save_pretrained(model_dir)

# Save the model config
config = {
    "architectures": ["BertForSTS"],
    "model_type": "bert",
    "tokenizer_class": "BertTokenizer",
    "pretrained_model_name_or_path": "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext"
}
config_path = os.path.join(model_dir, "config.json")
with open(config_path, 'w') as f:
    import json
    json.dump(config, f)

# Create the repository on Hugging Face
api = HfApi()
username = api.whoami(token=HF_TOKEN)["name"]
repo_name = "OrphanSimBERT_1"
repo_id = f"{username}/{repo_name}"

api.create_repo(repo_id, token=HF_TOKEN, exist_ok=True)

# Upload files to the repository
upload_file(
    path_or_fileobj=model_path,
    path_in_repo="pytorch_model.bin",
    repo_id=repo_id,
    token=HF_TOKEN
)
upload_file(
    path_or_fileobj=config_path,
    path_in_repo="config.json",
    repo_id=repo_id,
    token=HF_TOKEN
)

# Upload tokenizer files
for file_name in ["tokenizer_config.json", "vocab.txt"]:
    file_path = os.path.join(model_dir, file_name)
    if os.path.exists(file_path):
        upload_file(
            path_or_fileobj=file_path,
            path_in_repo=file_name,
            repo_id=repo_id,
            token=HF_TOKEN
        )

print("Model and tokenizer files have been uploaded to the repository.")

No GPU available, using the CPU instead.


OrphanSimBERT.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Model and tokenizer files have been uploaded to the repository.


In [ ]:
### pushing model
from transformers import BertConfig, BertModel
config = BertConfig()
model = BertModel(config)
model.push_to_hub("jabv/OrphanSimBERT")
# reload
model = BertModel.from_pretrained("jabv/OrphanSimBERT")

In [ ]:
### pushing tokenizer
from transformers import BertTokenizer
# Instantiate your tokenizer
tokenizer = AutoTokenizer.from_pretrained('microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext')
tokenizer_directory = "./tokenizer_directory"
# Save the tokenizer files to the directory
tokenizer.save_pretrained(tokenizer_directory)
tokenizer.push_to_hub("jabv/OrphanSimBERT_1")

# Example

In [ ]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 1.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
from sentence_transformers import SentenceTransformer, models
from transformers import BertTokenizer
import torch

tokenizer = BertTokenizer.from_pretrained("jabv/OrphanSimBERT_1")

class OrphanSimBERT(torch.nn.Module):
    def __init__(self):
        super(OrphanSimBERT, self).__init__()
        self.bert = models.Transformer('jabv/OrphanSimBERT_1')
        self.pooling_layer = models.Pooling(self.bert.get_word_embedding_dimension())
        self.sts_bert = SentenceTransformer(modules=[self.bert, self.pooling_layer])

    def forward(self, input_data):
        output = self.sts_bert(input_data)['sentence_embedding']
        return output

def predict_similarity(sentence_pair):
    test_input = tokenizer(sentence_pair, padding=True, truncation=True, return_tensors="pt")
    test_input['input_ids'] = test_input['input_ids']
    test_input['attention_mask'] = test_input['attention_mask']
    del test_input['token_type_ids']
    output = model(test_input)
    sim = torch.nn.functional.cosine_similarity(output[0], output[1], dim=0).item()
    return sim

model = OrphanSimBERT()

example_1 = [  'arterial thoracic outlet syndrome',
 '''The objective of this project is to study the contribution of hemorrheology to arterial and venous thrombosis using the arterio-venous (A-V)
   ''']
example_2=[ 'A cat is walking around a house.', 'A woman is peeling potato']

print(f"Predicted similarity score for example_1: {round(predict_similarity(example_1), 2)}")
print(f"Predicted similarity score for example_2: {round(predict_similarity(example_2), 2)}")

In [ ]:
example_1 = [  'arterial thoracic outlet syndrome',
 '''The objective of this project is to study the contribution of hemorrheology to arterial and venous thrombosis using the arterio-venous (A-V)
   ''']
print(f"Sentence 1: {example_1[0]}")
print(f"Sentence 2: {example_1[1]}")
print(f"Predicted similarity score: {round(predict_similarity(example_1), 2)}")

In [ ]:
def predict_similarity(sentence_pair):
    test_input = tokenizer(sentence_pair, padding=True, truncation=True, return_tensors="pt")
    test_input['input_ids'] = test_input['input_ids']
    test_input['attention_mask'] = test_input['attention_mask']
    del test_input['token_type_ids']
    output = model(test_input)
    sim = torch.nn.functional.cosine_similarity(output[0], output[1], dim=0).item()
    return sim
example_1=[ 'A cat is walking around a house.', 'A woman is peeling potato']
model = OrphanSimBERT()


print(f"Sentence 1: {example_1[0]}")
print(f"Sentence 2: {example_1[1]}")
print(f"Predicted similarity score: {round(predict_similarity(example_1), 2)}")